# Groundtruth evaluation for Llama-Index applications

Ground truth evaluation can be especially useful during early LLM experiments when you have a small set of example queries that are critical to get right. Ground truth evaluation works by comparing the similarity of an LLM response compared to its matching verified response.

This example walks through how to set up ground truth eval for a Llama-Index app.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/truera/trulens/blob/main/trulens_eval/examples/expositional/frameworks/llama_index/llama_index_groundtruth.ipynb)

### import from TruLens and Llama-Index

In [ ]:
# ! pip install trulens_eval==0.24.0 llama_index==0.10.11

In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.readers.web import SimpleWebPageReader
import openai

from trulens_eval import TruLlama, Feedback, Tru, feedback
from trulens_eval.feedback import GroundTruthAgreement
tru = Tru()

In [ ]:
tru.reset_database()

### Add API keys
For this quickstart, you will need Open AI and Huggingface keys

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "..."
openai.api_key = os.environ["OPENAI_API_KEY"]

### Create Simple LLM Application

This example uses LlamaIndex which internally uses an OpenAI LLM.

In [ ]:
documents = SimpleWebPageReader(html_to_text=True).load_data(
    ["http://paulgraham.com/worked.html"]
)
index = VectorStoreIndex.from_documents(documents)

query_engine = index.as_query_engine()

## Initialize Feedback Function(s)

In [ ]:
# Initialize OpenAI-based feedback function collection class:
openai = feedback.OpenAI()

In [ ]:
golden_set = [
    {"query": "What was the author's undergraduate major?", "response": "He didn't choose a major, and customized his courses."},
    {"query": "What company did the author start in 1995?", "response": "Viaweb, to make software for building online stores."},
    {"query": "Where did the author move in 1998 after selling Viaweb?", "response": "California, after Yahoo acquired Viaweb."},
    {"query": "What did the author do after leaving Yahoo in 1999?", "response": "He focused on painting and tried to improve his art skills."},
    {"query": "What program did the author start with Jessica Livingston in 2005?", "response": "Y Combinator, to provide seed funding for startups."}
]

In [ ]:
f_groundtruth = Feedback(GroundTruthAgreement(golden_set).agreement_measure, name = "Ground Truth Eval").on_input_output()

### Instrument the application with Ground Truth Eval

In [ ]:
tru_query_engine_recorder = TruLlama(query_engine,
    app_id='LlamaIndex_App1',
    feedbacks=[f_groundtruth],
)

### Run the application for all queries in the golden set

In [ ]:
#Run and evaluate on groundtruth questions
for pair in golden_set:
    with tru_query_engine_recorder as recording:
        llm_response = query_engine.query(pair['query'])
        print(llm_response)

### Explore with the TruLens dashboard

In [ ]:
tru.run_dashboard() # open a local streamlit app to explore

# tru.stop_dashboard() # stop if needed

### Or view results directly in your notebook

In [ ]:
records, feedback = tru.get_records_and_feedback(app_ids=[]) # pass an empty list of app_ids to get all
records.head()